In [ ]:
from gensim.models import Word2Vec
sentences = [[ ' this ' , ' is ' , ' the ' , ' first ' , ' sentence ' , ' for ' , ' word2vec ' ],
[ ' this ' , ' is ' , ' the ' , ' second ' , ' sentence ' ], [ ' yet ' , ' another ' , ' sentence ' ],
[ ' one ' , ' more ' , ' sentence ' ], [ ' and ' , ' the ' , ' final ' , ' sentence ' ]]
model = Word2Vec(sentences, min_count=1)
print(model)
words = list(model.wv.vocab)
print(words)
print(model[ ' sentence ' ])
model.save( ' model.bin ' )
new_model = Word2Vec.load( ' model.bin ' )


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = ' glove.6B.100d.txt '
word2vec_output_file = ' glove.6B.100d.txt.word2vec '
glove2word2vec(glove_input_file, word2vec_output_file)


In [ ]:
#Now we can load it and perform the same (king - man) + woman = ?
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = ' glove.6B.100d.txt.word2vec '
model = KeyedVectors.load_word2vec_format(filename, binary=False)
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=[ ' woman ' , ' king ' ], negative=[ ' man ' ], topn=1)
print(result)


In [ ]:
#Each text document is classified as positive 1 or negative 0.
docs = [ ' Well done! ' , ' Good work ' , ' Great effort ' , ' nice work ' , ' Excellent! ' , ' Weak ' ,
' Poor effort! ' , ' not good ' , ' poor work ' , ' Could have done better. ' ]
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
The sequences have different lengths and Keras prefers inputs to be vectorized and all inputs to have the same length. We will pad all input sequences to have the length of 4
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding= ' post ' )
print(padded_docs)



In [ ]:
#We flatten this to a one 32-element vector to pass on to the Dense output layer.
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation= ' sigmoid ' ))
model.compile(optimizer= ' adam ' , loss= ' binary_crossentropy ' , metrics=[ ' acc ' ])
model.summary()


In [ ]:
#Example of Learning an Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print( ' Accuracy: %f ' % (accuracy*100))


In [ ]:
#Example of Using Pre-Trained GloVe Embedding
The Keras Embedding layer can also use a word embedding learned elsewhere
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding= ' post ' )
print(padded_docs)
#Next, we need to load the entire GloVe word embedding file into memory as a dictionary of word to embedding array.
# load the whole embedding into memory
embeddings_index = dict()
f = open( ' glove.6B.100d.txt ' )
for line in f:
values = line.split()
word = values[0]
coefs = asarray(values[1:], dtype= ' float32 ' )
embeddings_index[word] = coefs
f.close()
print( ' Loaded %s word vectors. ' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
embedding_vector = embeddings_index.get(word)
if embedding_vector is not None:
embedding_matrix[i] = embedding_vector



# complete worked example

In [ ]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation= ' sigmoid ' ))
model.compile(optimizer= ' adam ' , loss= ' binary_crossentropy ' , metrics=[ ' acc ' ])
model.summary()
model.fit(padded_docs, labels, epochs=50, verbose=0)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print( ' Accuracy: %f ' % (accuracy*100))


In [1]:
from numpy import array
from sklearn.preprocessing import LabelEncoder
# define example
data = ['warm','hot','cold','luke warm','hot','cold','warm']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
inverted = label_encoder.inverse_transform(integer_encoded)[2]
print(inverted)


['warm' 'hot' 'cold' 'luke warm' 'hot' 'cold' 'warm']
[3 1 0 2 1 0 3]
cold
